In [1]:
#import libraries

from bs4 import BeautifulSoup
import numpy as np
import requests
import re
import pandas as pd


#Loop over all pages

data = pd.DataFrame([])
for year in range(2013,2016):
    for month in range(1, 13):
        if (month<10):
            url = 'https://en.tutiempo.net/climate/0{}-{}/ws-421820.html'.format(month, year)
                        
        else:
            url = 'https://en.tutiempo.net/climate/{}-{}/ws-421820.html'.format(month, year)

        retreived_data = requests.get(url).text

        soup = BeautifulSoup(retreived_data, "lxml")

        hiddenData = str(soup.find_all('style')[1])
        hiddenSpan = {}
        for group in re.findall(r'span\.(.+?)}',hiddenData):
            class_attr = group.split('span.')[-1].split('::')[0]
            content = group.split('"')[1]
            hiddenSpan[class_attr] = content

        climate_table = str(soup.find("table", attrs={"class": "medias mensuales numspan"}))   
        for k, v in hiddenSpan.items():
            climate_table = climate_table.replace('<span class="%s"></span>' %(k), hiddenSpan[k])


        df = pd.read_html(climate_table)[0]
        df['Year'] = year
        df['Month'] = month

        data = data.append(df, ignore_index = True)
            

In [2]:
data.to_csv('AQI_Data_combine.csv')